In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor

df = pd.read_csv('../Deliverable4/daan881_group4_dataset.cleaned.csv')

print(df.shape)
df.head()

(93697, 15)


,cve,date_reserved,date_published,cvss_v4_score,cvss_v4_severity,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_pre_v4_avg_score,cvss_pre_v4_severity
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,NaN,NaN,1.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,6.500000,3.0
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,NaN,NaN,1.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,7.200000,2.0
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,NaN,NaN,1.0,0.0,NaN,2.0,0.0,NaN,NaN,NaN,9.100000,1.0
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,5.3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.666667,4.0
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,NaN,NaN,1.0,1.0,NaN,2.0,0.0,NaN,NaN,NaN,0.000000,NaN


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93697 entries, 0 to 93696
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cve                    93697 non-null  object 
 1   date_reserved          93697 non-null  object 
 2   date_published         93697 non-null  object 
 3   cvss_v4_score          1773 non-null   float64
 4   cvss_v4_severity       1773 non-null   float64
 5   attack_vector          42260 non-null  float64
 6   attack_complexity      42260 non-null  float64
 7   attack_requirements    626 non-null    float64
 8   privileges_required    42260 non-null  float64
 9   user_interaction       42260 non-null  float64
 10  exploit_maturity       1797 non-null   float64
 11  epss                   93506 non-null  float64
 12  percentile             93506 non-null  float64
 13  cvss_pre_v4_avg_score  54926 non-null  float64
 14  cvss_pre_v4_severity   54899 non-null  float64
dtypes:

## Testing for more colinearity

In [26]:
def get_correlation(data, columns):
    reduced_data = data[data[[*columns]].notna().all(1)]
    reduced_data = reduced_data[[*columns]]

    return reduced_data.corr(), [variance_inflation_factor(reduced_data.values, i) for i in range(len(columns))]

correlation, vif = get_correlation(df, ["cvss_pre_v4_avg_score", "cvss_v4_score"])

correlation

,cvss_pre_v4_avg_score,cvss_v4_score
cvss_pre_v4_avg_score,1.000000,0.699596
cvss_v4_score,0.699596,1.000000


In [27]:
vif

[np.float64(28.58921100070732), np.float64(28.58921100070741)]

We can see that 0.699596 is pretty close to the normal threhold of 0.7 for highly colinear. Additionally, the VIF is ~29 which is much higher than the >10 threshold for strongly colinear. So, we will take the v4 value if present, pre v4 value if not.

In [28]:
df['cvss_score'] = np.where(df['cvss_v4_score'].notna(), 
                                  df['cvss_v4_score'], 
                                  df['cvss_pre_v4_avg_score'])

df = df.drop(columns=["cvss_pre_v4_avg_score"])
df = df.drop(columns=["cvss_v4_score"])

df.head()

,cve,date_reserved,date_published,cvss_v4_severity,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_pre_v4_severity,cvss_score
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,NaN,1.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,3.0,6.5
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,NaN,1.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,2.0,7.2
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,NaN,1.0,0.0,NaN,2.0,0.0,NaN,NaN,NaN,1.0,9.1
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.3
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,NaN,1.0,1.0,NaN,2.0,0.0,NaN,NaN,NaN,NaN,0.0


Now we can do the same checks for severity

In [29]:
correlation, vif = get_correlation(df, ["cvss_v4_severity", "cvss_pre_v4_severity"])

correlation

,cvss_v4_severity,cvss_pre_v4_severity
cvss_v4_severity,1.000000,0.640824
cvss_pre_v4_severity,0.640824,1.000000


In [30]:
vif

[np.float64(28.799960858922507), np.float64(28.799960858922507)]

We can see that the severities are highly colinear based on the correlation corefficients and vif. So, we can merge them down as well.

In [31]:
df['cvss_severity'] = np.where(df['cvss_v4_severity'].notna(), 
                                  df['cvss_v4_severity'], 
                                  df['cvss_pre_v4_severity'])

df = df.drop(columns=["cvss_v4_severity"])
df = df.drop(columns=["cvss_pre_v4_severity"])

df.head()

,cve,date_reserved,date_published,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_score,cvss_severity
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,1.0,0.0,NaN,0.0,0.0,NaN,NaN,NaN,6.5,3.0
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,1.0,0.0,NaN,1.0,0.0,NaN,NaN,NaN,7.2,2.0
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,1.0,0.0,NaN,2.0,0.0,NaN,NaN,NaN,9.1,1.0
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.3,3.0
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,1.0,1.0,NaN,2.0,0.0,NaN,NaN,NaN,0.0,NaN


In [32]:
correlation, vif = get_correlation(df, ["epss", "cvss_score"])

correlation

,epss,cvss_score
epss,1.000000,0.099826
cvss_score,0.099826,1.000000


We can see that epss and cvss_score are not correlated

In [33]:
df['epss'].isna().sum()

np.int64(191)

In [34]:
df['cvss_score'].isna().sum()

np.int64(38413)

In [35]:
df[df[['epss', 'cvss_score']].isna().all(axis=1)].shape[0]

0

Above we can see that every row has either a cvss score or a epss score. We will load the column with a mission value with -1.

In [36]:
df['epss'] = df['epss'].fillna(-1)
df['percentile'] = df['percentile'].fillna(-1)
df['cvss_score'] = df['cvss_score'].fillna(-1)
df['cvss_severity'] = df['cvss_severity'].fillna(-1)

df.head()

,cve,date_reserved,date_published,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_score,cvss_severity
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,1.0,0.0,NaN,0.0,0.0,NaN,-1.0,-1.0,6.5,3.0
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,1.0,0.0,NaN,1.0,0.0,NaN,-1.0,-1.0,7.2,2.0
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,1.0,0.0,NaN,2.0,0.0,NaN,-1.0,-1.0,9.1,1.0
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,5.3,3.0
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,1.0,1.0,NaN,2.0,0.0,NaN,-1.0,-1.0,0.0,-1.0


We should assume that if the attack vector is Nan that we take the worst case which would be a NETWORK attack because that would be the greatest threat without physical access. NETWORK maps to 1. So, we will update all NaN to 1 for attack_vector.

In [37]:
df['attack_vector'] = df['attack_vector'].fillna(1)

df.head()

,cve,date_reserved,date_published,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_score,cvss_severity
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,1.0,0.0,NaN,0.0,0.0,NaN,-1.0,-1.0,6.5,3.0
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,1.0,0.0,NaN,1.0,0.0,NaN,-1.0,-1.0,7.2,2.0
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,1.0,0.0,NaN,2.0,0.0,NaN,-1.0,-1.0,9.1,1.0
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,1.0,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,5.3,3.0
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,1.0,1.0,NaN,2.0,0.0,NaN,-1.0,-1.0,0.0,-1.0


We should also assume the worst case scenario for attack complexity which would be LOW (aka 0)

In [38]:
df['attack_complexity'] = df['attack_complexity'].fillna(0)

df.head()

,cve,date_reserved,date_published,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_score,cvss_severity
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,1.0,0.0,NaN,0.0,0.0,NaN,-1.0,-1.0,6.5,3.0
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,1.0,0.0,NaN,1.0,0.0,NaN,-1.0,-1.0,7.2,2.0
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,1.0,0.0,NaN,2.0,0.0,NaN,-1.0,-1.0,9.1,1.0
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,1.0,0.0,NaN,NaN,NaN,NaN,-1.0,-1.0,5.3,3.0
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,1.0,1.0,NaN,2.0,0.0,NaN,-1.0,-1.0,0.0,-1.0


The worst case scenario for attack_requirements would be NONE (aka 0). Meaning that there isn't any requirements required to make the attack.

In [39]:
df['attack_requirements'] = df['attack_requirements'].fillna(0)

df.head()

,cve,date_reserved,date_published,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_score,cvss_severity
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,1.0,0.0,0.0,0.0,0.0,NaN,-1.0,-1.0,6.5,3.0
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,1.0,0.0,0.0,1.0,0.0,NaN,-1.0,-1.0,7.2,2.0
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,1.0,0.0,0.0,2.0,0.0,NaN,-1.0,-1.0,9.1,1.0
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,1.0,0.0,0.0,NaN,NaN,NaN,-1.0,-1.0,5.3,3.0
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,1.0,1.0,0.0,2.0,0.0,NaN,-1.0,-1.0,0.0,-1.0


The worst case scenario for privleges_required would also be NONE (aka 2)

In [40]:
df['privileges_required'] = df['privileges_required'].fillna(2)

df.head()

,cve,date_reserved,date_published,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_score,cvss_severity
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,1.0,0.0,0.0,0.0,0.0,NaN,-1.0,-1.0,6.5,3.0
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,1.0,0.0,0.0,1.0,0.0,NaN,-1.0,-1.0,7.2,2.0
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,1.0,0.0,0.0,2.0,0.0,NaN,-1.0,-1.0,9.1,1.0
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,1.0,0.0,0.0,2.0,NaN,NaN,-1.0,-1.0,5.3,3.0
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,1.0,1.0,0.0,2.0,0.0,NaN,-1.0,-1.0,0.0,-1.0


The worst case scenario for user_interaction would be NONE (aka 0)

In [41]:
df['user_interaction'] = df['user_interaction'].fillna(0)

df.head()

,cve,date_reserved,date_published,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_score,cvss_severity
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,1.0,0.0,0.0,0.0,0.0,NaN,-1.0,-1.0,6.5,3.0
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,1.0,0.0,0.0,1.0,0.0,NaN,-1.0,-1.0,7.2,2.0
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,1.0,0.0,0.0,2.0,0.0,NaN,-1.0,-1.0,9.1,1.0
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,1.0,0.0,0.0,2.0,0.0,NaN,-1.0,-1.0,5.3,3.0
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,1.0,1.0,0.0,2.0,0.0,NaN,-1.0,-1.0,0.0,-1.0


The worst case for exploit maturity would be HIGH (aka 3)

In [42]:
df['exploit_maturity'] = df['exploit_maturity'].fillna(3)

df.head()

,cve,date_reserved,date_published,attack_vector,attack_complexity,attack_requirements,privileges_required,user_interaction,exploit_maturity,epss,percentile,cvss_score,cvss_severity
0,CVE-2024-8969,2024-09-18 04:19:44.810000+00:00,2024-09-18T06:53:53.016Z,1.0,0.0,0.0,0.0,0.0,3.0,-1.0,-1.0,6.5,3.0
1,CVE-2024-8957,2024-09-17 19:08:48.129000+00:00,2024-09-17T20:08:25.588Z,1.0,0.0,0.0,1.0,0.0,3.0,-1.0,-1.0,7.2,2.0
2,CVE-2024-8956,2024-09-17 19:08:47.005000+00:00,2024-09-17T19:59:27.205Z,1.0,0.0,0.0,2.0,0.0,3.0,-1.0,-1.0,9.1,1.0
3,CVE-2024-8951,2024-09-17 15:24:05.559000+00:00,2024-09-17T20:00:08.078Z,1.0,0.0,0.0,2.0,0.0,3.0,-1.0,-1.0,5.3,3.0
4,CVE-2024-47059,2024-09-17 13:41:00.585000+00:00,2024-09-18T21:19:26.951Z,1.0,1.0,0.0,2.0,0.0,3.0,-1.0,-1.0,0.0,-1.0


In [43]:
df.to_csv("daan881_group4_dataset.final.cleaned.csv", index=False)